# Notebook Objective and Setup

BGG03 is the scrubbing and cleaning of the various data obtained in notebooks BGG01 and BGG02. The following datasets are cleaned, constructed, or otherwise prepared for EDA and modeling.

    * Games
    * Mechanics
    * Subcategories
    * Designers
    * Artists
    * Publishers
    * Awards
    * Ratings Distribution
    * Comments
    * Ratings Matrix

## Package Imports

In [1]:
import pandas as pd
import numpy as np
import requests
import regex as re
import time
import os
import gc

# ignore warnings (gets rid of Pandas copy warnings)
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

# NLP tools
import spacy
nlp = spacy.load("en_core_web_sm")
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import word_tokenize

## Notebook Functions

In [2]:
def integer_reduce(data, columns, fill_value=0):
    '''
    Reduces an integer type to its smallest memory size type
    
    Inputs:
    data: dataframe to reduce
    columns: columns to reduce
    fill_value: fill value to use if none
    
    Returns: 
    data: dataframe with memory reduced data types
    '''
    for column in columns:
        print(column)
        data[column].fillna(fill_value, inplace=True)
        if (data[column].max() <= 127) & (data[column].min() >= -128):
            data[column] = data[column].astype('int8')
        elif (data[column].max() <= 32767) & (data[column].min() >= -32768):
            data[column] = data[column].astype('int16')
        elif (data[column].max() <= 2147483647) & (data[column].min() >= -2147483648):
            data[column] = data[column].astype('int32')
        
    return data

In [3]:
def text_block_processor(text):
    '''Takes a block of text. Divides block into sentences with words lemmatized.
    Sends each sentence to word processor. Concatenates all words into one string
    Otherwise returns string of cleaned and processed words from text block
    
    ARGUMENTS:
    block of text
    '''
    
    text = str(text)
    line = re.sub(r'[^a-zA-Z\s]', '', text).lower() # removes all special characters and numbers, and makes lower case
    tokens = nlp(line)
    words = []
    for token in tokens:
        if token.is_stop == False:
            token_preprocessed = token.lemma_
            if token_preprocessed != '': # only continues if returned word is not empty
                words.append(token_preprocessed) # appends word to list of words
    line = ' '.join(words)
    
    return line


In [4]:
def fix_numbers(x):
    '''
    Checks for numbers or strings
    If a string, strips off the "k" and multiply by 10000
    Sends back cleaned int
    '''
    
    if type(x) is int:
        return int(x)
    
    if str.endswith(x, 'k'):
        x = str(x).strip('k')
        new_num = int(float(x)*1000)
        return int(new_num)
    
    else: 
        return int(x)

In [5]:
def clean_ratings(id_num, game_ids_list):
    '''
    Loads and cleans a raw user ratings file
    Drops game ids not present in games file
    Drops users with fewer than 10 ratings
    
    Inputs:
    id_num: the appendation of the file to find the path
    game_ids_list: list of game ids in the games file
    
    Outputs:
    Cleaned user ratings file
    '''
    
    print('\nCleaning Frame #'+str(id_num))
    
    #load in raw users file according to id_num inputted
    users = pd.read_pickle('userid/user_ratings'+str(id_num)+'.pkl')
    
    # convert all datatypes to float
    float_converted = users.astype('float')
    
    # delete and clean up raw users file
    del users
    gc.collect()
    
    # create intersection between user file and game list ids
    cleaned = float_converted[float_converted.columns.intersection(game_ids_list)]
    
    # delete and clean up
    del float_converted
    gc.collect()
    
    # make a list of users with fewer than 10 user ratings
    sums = cleaned.count(axis=1)<10
    # get indices for the rows with fewer than 10 ratings
    drop_these = sums.loc[sums==True].index
    # drop the users with fewer than 10 ratings
    cleaned.drop(drop_these, axis=0, inplace=True)
    
    # print memory usage
    print(cleaned.info())
    
    # return cleaned file
    return cleaned

In [6]:
def create_ratings_file(start_file, end_file, game_ids_list):
    '''
    Puts together dataframes from a range of files
    Each file calls the clean_ratings function
    Then all files in range are concatenated
    
    Inputs:
    start_file: start of file name appendation
    end_file: end file name appendation
    game_ids_list: list of game ids in the games file
    
    Outputs:
    Cleaned and concatenated master file
    
    '''
    
    # make an empty dataframe
    master_file = pd.DataFrame()
    
    # for each number in the range from start to end:
    for id_num in np.arange(start_file,end_file+1,1):
        # clean the file calling clean_ratings
        cleaned_item = clean_ratings(id_num, game_ids_list)
        # append the file to the dataframe
        master_file = pd.concat([master_file, cleaned_item], axis=0)
    
    # clean up
    del cleaned_item
    gc.collect()
    
    return master_file

## Games

In [7]:
games = pd.read_pickle('data_dirty/games.pkl')
games

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,ComAgeRec,LanguageEase,BestPlayers,GoodPlayers,NumOwned,NumWant,NumWish,NumWeightVotes,MfgPlaytime,ComMinPlaytime,ComMaxPlaytime,MfgAgeRec,NumUserRatings,NumComments,NumAlternates,NumExpansions,NumAwards,NumImplementations,NumFans,NumPageViews,RulesPosts,TotalPosts,IsExpansion,IsReimplementation,Family,Theme,Category,Kickstarted,ImagePath,Rank:boardgame,Rank:thematic,Rank:strategygames,Rank:wargames,Rank:familygames,Rank:cgs,Rank:abstracts,Rank:partygames,Rank:childrensgames,Rank:rpgitem,Rank:boardgameaccessory,Rank:videogame,Rank:amiga,Rank:commodore64,Rank:arcade,Rank:atarist
0,174430,Gloomhaven,Gloomhaven is a game of Euro-inspired tactica...,2017,3.8726,8.76029,8.52385,1.63358,1,4,12.784946,4.152542,3,"[1, 2, 3, 4]",74975,1375,17276,2009,120,60,120,14,46228,8308,6,12,30,0,7718,10816029,6326,14752,0,0,Gloomhaven,NaN,Dungeon Crawl,1,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZ...,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,161936,Pandemic Legacy: Season 1,Pandemic Legacy is a co-operative campaign gam...,2015,2.8331,8.60116,8.45000,1.56368,2,4,11.354037,4.077778,4,"[2, 3, 4]",68974,840,11544,1210,60,60,60,13,44043,6682,10,0,30,2,2952,3450336,1229,3086,0,1,Pandemic,NaN,NaN,NaN,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...,2.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,224517,Brass: Birmingham,Brass: Birmingham is an economic strategy game...,2018,3.9038,8.66907,8.41066,1.24331,2,4,13.226190,1.035714,3,"[2, 3, 4]",35448,1528,11277,1143,120,60,120,14,23776,3610,6,0,18,1,1902,1974872,387,1028,0,1,Brass,Canals,NaN,1,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,3.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,167791,Terraforming Mars,"In the 2400s, mankind begins to terraform the ...",2016,3.2429,8.42155,8.27751,1.38628,1,5,11.891156,3.380952,3,"[1, 2, 3, 4]",97330,2061,18842,2944,120,120,120,12,71474,10452,14,22,39,2,6372,6285054,1155,4313,0,0,Terraforming Mars,NaN,NaN,NaN,https://cf.geekdo-images.com/wg9oOLcsKvDesSUdZ...,4.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,291457,Gloomhaven: Jaws of the Lion,Gloomhaven: Jaws of the Lion is a standalone g...,2020,3.5649,8.72198,8.25902,1.42169,1,4,12.051948,4.000000,2,"[1, 2, 3, 4]",33444,477,6250,485,120,30,120,14,13939,2149,6,0,11,0,1649,1471587,1073,2062,0,0,Gloomhaven,NaN,Dungeon Crawl,NaN,https://cf.geekdo-images.com/_HhIdavYW-hid20Iq...,5.0,3.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21403,7316,Bingo,A classic party game in which players cover pl...,1530,1.0439,2.87934,3.96295,1.76399,2,99,4.720000,1.062500,3,[3],1653,2,27,205,60,60,60,5,2257,707,72,0,0,0,10,57207,0,12,0,0,Classic (Schmidt Spiele),NaN,NaN,NaN,https://cf.geekdo-images.com/7xqN6StcQz1FoGplo...,21478.0,NaN,NaN,NaN,NaN,NaN,NaN,631.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21404,5048,Candy Land,Created by Eleanor Abbott in the early 1940's ...,1949,1.1012,3.18388,3.79668,1.72104,2,4,3.325000,1.052632,4,"[2, 3, 4]",6109,4,66,346,30,30,30,3,4178,1543,10,0,0,3,36,321486,3,108,0,0,NaN,Food / Cooking,NaN,NaN,https://cf.geekdo-images.com/97n-BYkjnFiHAhqUz...,21479.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,873.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21405,5432,Chutes and Ladders,Traditional game from ancient India was brough...,-200,1.0195,2.86610,3.61367,1.64312,2,6,3.357143,1.000000,4,"[2, 3, 4, 5]",4705,4,58,308,30,30,30,3,3967,1318,162,0,0,0,22,252400,0,55,0,0,GoPlay,Circus,NaN,NaN,https://cf.geekdo-images.com/P1qJDS_DFTtP_FrpW...,21480.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,874.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21406,11901,Tic-Tac-Toe,A very old game where each player attempts to ...,-1300,1.1697,2.69687,3.57174,1.98415,2,2,4.181818,1.0357

In [8]:
# Get info, make note of datatypes and memory usage
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21408 entries, 0 to 21407
Data columns (total 55 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   BGGId                    21408 non-null  int64  
 1   Name                     21408 non-null  object 
 2   Description              21408 non-null  object 
 3   YearPublished            21408 non-null  int64  
 4   GameWeight               21408 non-null  float64
 5   AvgRating                21408 non-null  float64
 6   BayesAvgRating           21408 non-null  float64
 7   StdDev                   21408 non-null  float64
 8   MinPlayers               21408 non-null  int64  
 9   MaxPlayers               21408 non-null  int64  
 10  ComAgeRec                16010 non-null  float64
 11  LanguageEase             15642 non-null  float64
 12  BestPlayers              1929 non-null   object 
 13  GoodPlayers              21408 non-null  object 
 14  NumOwned              

In [9]:
drops = games.loc[games['BGGId']==0].index # get indices of any games with no BGGId
games.drop(games.loc[drops].index, axis=0, inplace=True) # drop games with no BGGId

In [10]:
games.drop_duplicates(subset='BGGId', keep='first', inplace=True) # drop duplicate entires

# drop non-boardgame related information
games.drop(['Rank:rpgitem', 'Rank:boardgameaccessory', 'Rank:videogame', 'Rank:amiga', 'Rank:commodore64', 'Rank:arcade', 'Rank:atarist'], axis=1, inplace=True)

games.shape # check shape of file

(21369, 48)

In [11]:
# what are the columns?
games.columns

Index(['BGGId', 'Name', 'Description', 'YearPublished', 'GameWeight',
       'AvgRating', 'BayesAvgRating', 'StdDev', 'MinPlayers', 'MaxPlayers',
       'ComAgeRec', 'LanguageEase', 'BestPlayers', 'GoodPlayers', 'NumOwned',
       'NumWant', 'NumWish', 'NumWeightVotes', 'MfgPlaytime', 'ComMinPlaytime',
       'ComMaxPlaytime', 'MfgAgeRec', 'NumUserRatings', 'NumComments',
       'NumAlternates', 'NumExpansions', 'NumAwards', 'NumImplementations',
       'NumFans', 'NumPageViews', 'RulesPosts', 'TotalPosts', 'IsExpansion',
       'IsReimplementation', 'Family', 'Theme', 'Category', 'Kickstarted',
       'ImagePath', 'Rank:boardgame', 'Rank:thematic', 'Rank:strategygames',
       'Rank:wargames', 'Rank:familygames', 'Rank:cgs', 'Rank:abstracts',
       'Rank:partygames', 'Rank:childrensgames'],
      dtype='object')

In [12]:
games['BestPlayers'].fillna(0, inplace=True) # Fill NaN on BestPlayers
games['BestPlayers'].unique() # What are the unique BestPlayers entries?

array(['3', '4', '2', '6', '1', '5', '7', '8', 0, '3+', '14', '15', '12',
       '0+', '9', '13', '11'], dtype=object)

In [13]:
# Clean up Best Players so all are integers
games.loc[(games['BestPlayers']=='3+'), 'BestPlayers'] = 3
games.loc[(games['BestPlayers']=='0+'), 'BestPlayers'] = 4

# change dtype on BestPlayers to int8
games['BestPlayers'] = games['BestPlayers'].astype('int8')

In [14]:
# Add Categories with binary flags
games.loc[games['Rank:thematic'].notna(), 'Cat:Thematic'] = int(1)
games.loc[games['Rank:strategygames'].notna(), 'Cat:Strategy'] = int(1)
games.loc[games['Rank:wargames'].notna(), 'Cat:War'] = int(1)
games.loc[games['Rank:familygames'].notna(), 'Cat:Family'] = int(1)
games.loc[games['Rank:cgs'].notna(), 'Cat:CGS'] = int(1)
games.loc[games['Rank:abstracts'].notna(), 'Cat:Abstract'] = int(1)
games.loc[games['Rank:partygames'].notna(), 'Cat:Party'] = int(1)
games.loc[games['Rank:childrensgames'].notna(), 'Cat:Childrens'] = int(1)

In [15]:
# prepare different column sets for memory integer reduction

# integer reduction with fill_values of 0
int_columns=['BGGId', 'YearPublished', 'MinPlayers', 'MaxPlayers', 'NumOwned',
       'NumWant', 'NumWish', 'NumWeightVotes', 'MfgPlaytime', 'ComMinPlaytime',
       'ComMaxPlaytime','MfgAgeRec', 'NumUserRatings', 'NumComments',
       'NumAlternates', 'NumExpansions', 'NumAwards', 'NumImplementations',
       'NumFans', 'NumPageViews', 'RulesPosts', 'TotalPosts', 'IsExpansion',
       'IsReimplementation', 'Kickstarted', 'Cat:Thematic', 'Cat:Strategy', 
        'Cat:War', 'Cat:Family', 'Cat:CGS', 'Cat:Abstract', 'Cat:Party', 'Cat:Childrens']

# integer reduction with fill_values of 21369 (lower is better on these)
ranks = ['Rank:boardgame', 'Rank:thematic', 'Rank:strategygames',
       'Rank:wargames', 'Rank:familygames', 'Rank:cgs', 'Rank:abstracts',
       'Rank:partygames', 'Rank:childrensgames']

# call integer_reduce on the sets
games = integer_reduce(games, int_columns, fill_value=0)

games = integer_reduce(games, ranks, fill_value=21369)

games.info() # recheck data types and memory usage

BGGId
YearPublished
MinPlayers
MaxPlayers
NumOwned
NumWant
NumWish
NumWeightVotes
MfgPlaytime
ComMinPlaytime
ComMaxPlaytime
MfgAgeRec
NumUserRatings
NumComments
NumAlternates
NumExpansions
NumAwards
NumImplementations
NumFans
NumPageViews
RulesPosts
TotalPosts
IsExpansion
IsReimplementation
Kickstarted
Cat:Thematic
Cat:Strategy
Cat:War
Cat:Family
Cat:CGS
Cat:Abstract
Cat:Party
Cat:Childrens
Rank:boardgame
Rank:thematic
Rank:strategygames
Rank:wargames
Rank:familygames
Rank:cgs
Rank:abstracts
Rank:partygames
Rank:childrensgames
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21369 entries, 0 to 21407
Data columns (total 56 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   BGGId                21369 non-null  int32  
 1   Name                 21369 non-null  object 
 2   Description          21369 non-null  object 
 3   YearPublished        21369 non-null  int16  
 4   GameWeight           21369 non-null  float64
 5   Avg

In [16]:
# any games with no name listed?
games.loc[games['Name']==0]

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,ComAgeRec,LanguageEase,BestPlayers,GoodPlayers,NumOwned,NumWant,NumWish,NumWeightVotes,MfgPlaytime,ComMinPlaytime,ComMaxPlaytime,MfgAgeRec,NumUserRatings,NumComments,NumAlternates,NumExpansions,NumAwards,NumImplementations,NumFans,NumPageViews,RulesPosts,TotalPosts,IsExpansion,IsReimplementation,Family,Theme,Category,Kickstarted,ImagePath,Rank:boardgame,Rank:thematic,Rank:strategygames,Rank:wargames,Rank:familygames,Rank:cgs,Rank:abstracts,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens


In [17]:
# if so, drop them and reset index
empty_games = list(games.loc[games['Name']==0].index)
games.drop(games.index[empty_games], inplace=True)
games.reset_index(inplace=True, drop=True)

In [18]:
# Drop all games that are not yet released, then reset index
not_released = list(games.loc[games['YearPublished']>2021].index)
games.drop(games.index[not_released], inplace=True)
games.reset_index(inplace=True, drop=True)

In [19]:
# process the Description column text
games['Description'] = games['Description'].apply(lambda x: text_block_processor(x))

KeyboardInterrupt: 

In [20]:
# All games with over 12 players are set at 13 players.
games.loc[games['MaxPlayers']>12, 'MaxPlayers'] = 13

In [21]:
# Games with min players of 0, we will set their min players = 2
games.loc[games['MinPlayers']<1, 'MaxPlayers'] = 2

In [22]:
# Cleaned and prepared games frame
games

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,ComAgeRec,LanguageEase,BestPlayers,GoodPlayers,NumOwned,NumWant,NumWish,NumWeightVotes,MfgPlaytime,ComMinPlaytime,ComMaxPlaytime,MfgAgeRec,NumUserRatings,NumComments,NumAlternates,NumExpansions,NumAwards,NumImplementations,NumFans,NumPageViews,RulesPosts,TotalPosts,IsExpansion,IsReimplementation,Family,Theme,Category,Kickstarted,ImagePath,Rank:boardgame,Rank:thematic,Rank:strategygames,Rank:wargames,Rank:familygames,Rank:cgs,Rank:abstracts,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens
0,174430,Gloomhaven,Gloomhaven is a game of Euro-inspired tactica...,2017,3.8726,8.76029,8.52385,1.63358,1,4,12.784946,4.152542,3,"[1, 2, 3, 4]",74975,1375,17276,2009,120,60,120,14,46228,8308,6,12,30,0,7718,10816029,6326,14752,0,0,Gloomhaven,NaN,Dungeon Crawl,1,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZ...,1,1,1,21369,21369,21369,21369,21369,21369,1,1,0,0,0,0,0,0
1,161936,Pandemic Legacy: Season 1,Pandemic Legacy is a co-operative campaign gam...,2015,2.8331,8.60116,8.45000,1.56368,2,4,11.354037,4.077778,4,"[2, 3, 4]",68974,840,11544,1210,60,60,60,13,44043,6682,10,0,30,2,2952,3450336,1229,3086,0,1,Pandemic,NaN,NaN,0,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...,2,2,3,21369,21369,21369,21369,21369,21369,1,1,0,0,0,0,0,0
2,224517,Brass: Birmingham,Brass: Birmingham is an economic strategy game...,2018,3.9038,8.66907,8.41066,1.24331,2,4,13.226190,1.035714,3,"[2, 3, 4]",35448,1528,11277,1143,120,60,120,14,23776,3610,6,0,18,1,1902,1974872,387,1028,0,1,Brass,Canals,NaN,1,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,3,21369,2,21369,21369,21369,21369,21369,21369,0,1,0,0,0,0,0,0
3,167791,Terraforming Mars,"In the 2400s, mankind begins to terraform the ...",2016,3.2429,8.42155,8.27751,1.38628,1,5,11.891156,3.380952,3,"[1, 2, 3, 4]",97330,2061,18842,2944,120,120,120,12,71474,10452,14,22,39,2,6372,6285054,1155,4313,0,0,Terraforming Mars,NaN,NaN,0,https://cf.geekdo-images.com/wg9oOLcsKvDesSUdZ...,4,21369,6,21369,21369,21369,21369,21369,21369,0,1,0,0,0,0,0,0
4,291457,Gloomhaven: Jaws of the Lion,Gloomhaven: Jaws of the Lion is a standalone g...,2020,3.5649,8.72198,8.25902,1.42169,1,4,12.051948,4.000000,2,"[1, 2, 3, 4]",33444,477,6250,485,120,30,120,14,13939,2149,6,0,11,0,1649,1471587,1073,2062,0,0,Gloomhaven,NaN,Dungeon Crawl,0,https://cf.geekdo-images.com/_HhIdavYW-hid20Iq...,5,3,5,21369,21369,21369,21369,21369,21369,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21288,16398,War,War can be played with a standard 52 card deck...,0,1.0000,2.29912,4.01489,1.64441,2,2,4.357143,1.000000,0,[],446,2,7,128,30,30,30,4,1392,486,31,0,0,1,8,64834,2,23,0,0,NaN,NaN,NaN,0,https://cf.geekdo-images.com/CeBpBleLUkV6uM-98...,21477,21369,21369,21369,21369,21369,21369,21369,876,0,0,0,0,0,0,0,1
21289,7316,Bingo,A classic party game in which players cover pl...,1530,1.0439,2.87934,3.96295,1.76399,2,13,4.720000,1.062500,3,[3],1653,2,27,205,60,60,60,5,2257,707,72,0,0,0,10,57207,0,12,0,0,Classic (Schmidt Spiele),NaN,NaN,0,https://cf.geekdo-images.com/7xqN6StcQz1FoGplo...,21478,21369,21369,21369,21369,21369,21369,631,21369,0,0,0,0,0,0,1,0
21290,5048,Candy Land,Created by Eleanor Abbott in the early 1940's ...,1949,1.1012,3.18388,3.79668,1.72104,2,4,3.325000,1.052632,4,"[2, 3, 4]",6109,4,66,346,30,30,30,3,4178,1543,10,0,0,3,36,321486,3,108,0,0,NaN,Food / Cooking,NaN,0,https://cf.geekdo-images.com/97n-BYkjnFiHAhqUz...,21479,21369,21369,21369,21369,21369,21369,21369,873,0,0,0,0,0,0,0,1
21291,5432,Chutes and Ladders,Traditional game from ancient India was brough...,-200,1.0195,2.86610,3.61367,1.64312,2,6,3.357143,1.000000,4,"[2, 3, 4, 5]",4705,4,58,308,30,30,30,3,3967,1318,162,0,0,0,22,252400,0,55,0,0,GoP

In [ ]:
# save to file
#games.to_pickle('data_cleaned/games.pkl')

## Mechanics and Subcategories

### Load and Clean Mechanics

In [24]:
# Load mechanics and check memory usage
mechanics = pd.read_pickle('data_dirty/mechanics.pkl')
mechanics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21407 entries, 0 to 21406
Columns: 185 entries, BGGId to Impulse Movement
dtypes: float64(184), int32(1)
memory usage: 30.1 MB


In [25]:
mechanics.drop_duplicates(subset='BGGId', keep='first', inplace=True)  # drop duplicates
mechanics.fillna(0, inplace=True) # fill nan
mechanics.drop(mechanics.index[empty_games], inplace=True) # drop any empty games as identified in games section
mechanics.reset_index(inplace=True, drop=True) # reset index
mechanics.drop(mechanics.index[not_released], inplace=True) # drop games not released as identified in games section
mechanics.reset_index(inplace=True, drop=True) # reset index
mechanics.info() # check memory usage

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21293 entries, 0 to 21292
Columns: 185 entries, BGGId to Impulse Movement
dtypes: float64(184), int32(1)
memory usage: 30.0 MB


In [26]:
# Clean up mechanics
# Here we are using our domain knowledge to compact several different catogories into one

auction_list = ['Auction: Dexterity','Auction: Dutch','Auction: Dutch Priority',
                'Auction: Fixed Placement','Auction: English','Auction: Once Around','Auction: Sealed Bid',
                'Auction: Turn Order Until Pass','Multiple-Lot Auction','Closed Economy Auction','Selection Order Bid',
                'Constrained Bidding']

turn_order_list = ['Turn Order: Auction','Turn Order: Claim Action','Turn Order: Pass Order',
                   'Turn Order: Progressive','Turn Order: Random','Turn Order: Role Order','Turn Order: Stat-Based']

dumb_physical_list = ['Acting','Hot Potato','Singing','Rock-Paper-Scissors']

drafting = ['Card Drafting']

legacy = ['Legacy']

worker_placement = ['Worker Placement with Dice Workers','Worker Placement, Different Worker Types'] #'Worker Placement',

dexterity = ['Stacking and Balancing', 'Flicking']

# compacting categories here:

for item in worker_placement:
    mechanics.loc[mechanics[item]==1, 'Worker Placement'] = int(1)
    mechanics.drop([item], axis=1, inplace=True)

for item in auction_list:
    mechanics.loc[mechanics[item]==1, 'Auction/Bidding'] = int(1)
    mechanics.drop([item], axis=1, inplace=True)

mechanics['Dexterity'] = int(0)
for item in dexterity:
    mechanics.loc[mechanics[item]==1, 'Dexterity'] = int(1)
    mechanics.drop([item], axis=1, inplace=True)
    
mechanics['Physical'] = int(0)
for item in dumb_physical_list:
    mechanics.loc[mechanics[item]==1, 'Physical'] = int(1)
    mechanics.drop([item], axis=1, inplace=True)
    
mechanics.loc[mechanics['Card Drafting']==1, 'Drafting'] = int(1)

mechanics.loc[mechanics['Legacy']==1, 'Legacy Game'] = int(1)

mechanics.drop(turn_order_list, axis=1, inplace=True)
mechanics.drop(['Card Drafting','Legacy'], axis=1, inplace=True)

In [31]:
mechanics

,BGGId,Action Queue,Action Retrieval,Campaign / Battle Card Driven,Card Play Conflict Resolution,Communication Limits,Cooperative Game,Critical Hits and Failures,Deck Construction,"Deck, Bag, and Pool Building",Grid Movement,Hand Management,Hexagon Grid,Legacy Game,Modular Board,Once-Per-Game Abilities,Scenario / Mission / Campaign Game,Simultaneous Action Selection,Solo / Solitaire Game,Storytelling,Variable Player Powers,Action Points,Point to Point Movement,Set Collection,Trading,Income,Loans,Market,Network and Route Building,Score-and-Reset Game,Tech Trees / Tech Tracks,Variable Set-up,Drafting,End Game Bonuses,TableauBuilding,Take That,Tile Placement,Line of Sight,Action Drafting,Area Majority / Influence,Area-Impulse,Dice Rolling,Follow,King of the Hill,Variable Phase Order,Voting,Victory Points as a Resource,Area Movement,Delayed Purchase,Team-Based Game,Auction/Bidding,Events,Hidden Movement,Movement Points,Simulation,Ownership,Rondel,Track Movement,Action/Event,Advantage Token,Sudden Death Ending,Tug of War,Force Commitment,Narrative Choice / Paragraph,Grid Coverage,Layering,Increase Value of Unchosen Resources,Hidden Roles,Player Elimination,Semi-Cooperative Game,Traitor Game,Automatic Resource Growth,Push Your Luck,Worker Placement,Role Playing,Stat Check Resolution,Contracts,Race,Hidden Victory Points,Memory,Enclosure,Stock Holding,Pick-up and Deliver,Map Addition,Die Icon Resolution,Resource to Move,Trick-taking,Move Through Deck,Bias,Catch the Leader,Programmed Movement,Slide/Push,Commodity Speculation,Square Grid,Moving Multiple Units,Alliances,Kill Steal,Passed Action Token,Pattern Building,Investment,Secret Unit Deployment,Highest-Lowest Scoring,Mancala,Lose a Turn,Betting and Bluffing,Negotiation,Time Track,Connections,Targeted Clues,Order Counters,Bingo,Line Drawing,Paper-and-Pencil,Deduction,Movement Template,Multiple Maps,Re-rolling and Locking,Roll / Spin and Move,Finale Ending,Roles with Asymmetric Information,Command Cards,"I Cut, You Choose",Prisoner's Dilemma,Interrupts,Real-Time,Ladder Climbing,Predictive Bid,Three Dimensional Movement,Zone of Control,Random Production,Relative Movement,Cube Tower,Bribery,Map Deformation,Elapsed Real Time Ending,Melding and Splaying,Pieces as Map,Pattern Movement,Static Capture,Pattern Recognition,Minimap Resolution,Map Reduction,Ratio / Combat Results Table,Single Loser Game,Chaining,Different Dice Movement,Measurement Movement,Action Timer,Physical Removal,Induction,Speed Matching,Player Judge,Crayon Rail System,Chit-Pull System,Matching,Impulse Movement,Dexterity,Physical
0,174430,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,161936,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,224517,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,167791,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,291457,1,1,1,0,1,1,1,1,0,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

### Load and Clean Subcategories

In [32]:
# load subcategories file and check memory usage

indices = list(games['BGGId'])
subcategories = pd.read_pickle('data_dirty/subcategories.pkl')
subcategories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24577 entries, 0 to 24576
Data columns (total 84 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   BGGId                       24577 non-null  object 
 1   Adventure                   1257 non-null   float64
 2   Exploration                 956 non-null    float64
 3   Fantasy                     2870 non-null   float64
 4   Fighting                    1783 non-null   float64
 5   Miniatures                  1188 non-null   float64
 6   Environmental               220 non-null    float64
 7   Medical                     97 non-null     float64
 8   Economic                    1702 non-null   float64
 9   Industry / Manufacturing    309 non-null    float64
 10  Transportation              457 non-null    float64
 11  Science Fiction             1860 non-null   float64
 12  Space Exploration           308 non-null    float64
 13  Territory Building          507

In [33]:
subcategories.drop_duplicates(subset='BGGId', keep='first', inplace=True) # drop duplicates
subcategories.fillna(0, inplace=True) # fill nan
subcategories.set_index('BGGId', drop=True, inplace=True) # drop bad games
subcategories = subcategories.loc[indices] # use only indices in games file
subcategories.reset_index(inplace=True) # reset index
subcategories

,BGGId,Adventure,Exploration,Fantasy,Fighting,Miniatures,Environmental,Medical,Economic,Industry / Manufacturing,Transportation,Science Fiction,Space Exploration,Territory Building,Civilization,Negotiation,Political,Wargame,Civil War,Movies / TV / Radio theme,Card Game,Novel-based,Age of Reason,Mythology,Renaissance,American West,Animals,Modern Warfare,Dice,Medieval,Ancient,City Building,Nautical,Post-Napoleonic,Horror,Educational,Puzzle,Collectible Components,Farming,Religious,Travel,Murder/Mystery,Pirates,Comic Book / Strip,Mature / Adult,Video Game Theme,Spies/Secret Agents,Abstract Strategy,Bluffing,Action / Dexterity,Arabian,Prehistoric,Deduction,Trains,Party Game,Word Game,Aviation / Flight,Zombies,World War II,Racing,Pike and Shot,World War I,Real-time,Humor,Print & Play,Sports,Electronic,Maze,Mafia,Expansion for Base-game,American Indian Wars,Napoleonic,American Revolutionary War,Children's Game,Memory,Vietnam War,American Civil War,Math,Number,Trivia,Music,Korean War,Game System,Book
0,174430,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,161936,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,224517,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,167791,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,291457,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21288,16398,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21289,7316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
21290,5048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0

### Clean/Combine Mechanics and Subcategories

Manually cleaning up Subcategories. This section on BGG has a lot of "catch-all" concept that involve theming, mechanics (which should be in the mechanics section), and large subcategories that should be alone. We use our domain knowledge to clean this section.

In [36]:
# print our subcategories so we can prepare to sort them
subcategories.columns

Index(['BGGId', 'Adventure', 'Exploration', 'Fantasy', 'Fighting',
       'Miniatures', 'Environmental', 'Medical', 'Economic',
       'Industry / Manufacturing', 'Transportation', 'Science Fiction',
       'Space Exploration', 'Territory Building', 'Civilization',
       'Negotiation', 'Political', 'Wargame', 'Civil War',
       'Movies / TV / Radio theme', 'Card Game', 'Novel-based',
       'Age of Reason', 'Mythology', 'Renaissance', 'American West', 'Animals',
       'Modern Warfare', 'Dice', 'Medieval', 'Ancient', 'City Building',
       'Nautical', 'Post-Napoleonic', 'Horror', 'Educational', 'Puzzle',
       'Collectible Components', 'Farming', 'Religious', 'Travel',
       'Murder/Mystery', 'Pirates', 'Comic Book / Strip', 'Mature / Adult',
       'Video Game Theme', 'Spies/Secret Agents', 'Abstract Strategy',
       'Bluffing', 'Action / Dexterity', 'Arabian', 'Prehistoric', 'Deduction',
       'Trains', 'Party Game', 'Word Game', 'Aviation / Flight', 'Zombies',
       'World W

In [38]:
# picking the items that will go under "themes"
subcats_themes = ['BGGId', 'Adventure', 'Fantasy', 'Fighting', 'Environmental', 'Medical', 'Economic',
       'Industry / Manufacturing', 'Transportation', 'Science Fiction',
       'Space Exploration', 'Civilization', 'Civil War',
       'Movies / TV / Radio theme', 'Novel-based',
       'Age of Reason', 'Mythology', 'Renaissance', 'American West', 'Animals',
       'Modern Warfare', 'Medieval', 'Ancient','Nautical', 'Post-Napoleonic', 'Horror',
         'Farming', 'Religious', 'Travel',
       'Murder/Mystery', 'Pirates', 'Comic Book / Strip', 'Mature / Adult',
       'Video Game Theme', 'Spies/Secret Agents','Arabian', 'Prehistoric',
                 'Trains','Aviation / Flight', 'Zombies',
       'World War II', 'Racing', 'Pike and Shot', 'World War I','Humor','Sports',
                 'Mafia','American Indian Wars', 'Napoleonic',
       'American Revolutionary War','Vietnam War', 'American Civil War','Number', 'Trivia',
       'Music', 'Korean War', 'City Building', 'Political', 'Math', 'Maze',]

In [39]:
# picking the items that will stay as subcategories
subcats_subcategories = ['BGGId', 'Exploration', 'Miniatures', 'Territory Building', 'Card Game',
             'Educational', 'Puzzle','Collectible Components',
        'Word Game','Print & Play','Electronic']

In [40]:
# looking at our mechanics in alphabetical order so we can see where to roll in the subcategory mechanics
sorted(mechanics.columns)

['Action Drafting',
 'Action Points',
 'Action Queue',
 'Action Retrieval',
 'Action Timer',
 'Action/Event',
 'Advantage Token',
 'Alliances',
 'Area Majority / Influence',
 'Area Movement',
 'Area-Impulse',
 'Auction/Bidding',
 'Automatic Resource Growth',
 'BGGId',
 'Betting and Bluffing',
 'Bias',
 'Bingo',
 'Bribery',
 'Campaign / Battle Card Driven',
 'Card Play Conflict Resolution',
 'Catch the Leader',
 'Chaining',
 'Chit-Pull System',
 'Command Cards',
 'Commodity Speculation',
 'Communication Limits',
 'Connections',
 'Contracts',
 'Cooperative Game',
 'Crayon Rail System',
 'Critical Hits and Failures',
 'Cube Tower',
 'Deck Construction',
 'Deck, Bag, and Pool Building',
 'Deduction',
 'Delayed Purchase',
 'Dexterity',
 'Dice Rolling',
 'Die Icon Resolution',
 'Different Dice Movement',
 'Drafting',
 'Elapsed Real Time Ending',
 'Enclosure',
 'End Game Bonuses',
 'Events',
 'Finale Ending',
 'Follow',
 'Force Commitment',
 'Grid Coverage',
 'Grid Movement',
 'Hand Managemen

In [41]:
# drop subcategories that are none of the things
subcategories.drop(['Expansion for Base-game',  'Game System', 'Book'], axis=1, inplace=True)

In [42]:
# for mechanics that are mistakenly in the subcategories frame, make sure those mechanics are in the mechanics frame instead
# remove them from the subcategories frame after moving them over

indices = list(subcategories.loc[subcategories['Negotiation']==1].index)
mechanics.loc[indices, 'Negotiation']=1
subcategories.drop('Negotiation', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Action / Dexterity']==1].index)
mechanics.loc[indices, 'Dexterity']=1
subcategories.drop('Action / Dexterity', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Dice']==1].index)
mechanics.loc[indices, 'Dice Rolling']=1
subcategories.drop('Dice', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Bluffing']==1].index)
mechanics.loc[indices, 'Betting and Bluffing']=1
subcategories.drop('Bluffing', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Real-time']==1].index)
mechanics.loc[indices, 'Real-Time']=1
subcategories.drop('Real-time', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Memory']==1].index)
mechanics.loc[indices, 'Memory']=1
subcategories.drop('Memory', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Deduction']==1].index)
mechanics.loc[indices, 'Deduction']=1
subcategories.drop('Deduction', axis=1, inplace=True)


In [43]:
# or for larger categories in the subcategories frame, move those to the games frame
# remove them from the subcategories frame after moving them over

indices = list(subcategories.loc[subcategories['Wargame']==1].index)
games.loc[indices, 'Cat:War']=1
subcategories.drop('Wargame', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories["Children's Game"]==1].index)
games.loc[indices, 'Cat:Childrens']=1
subcategories.drop("Children's Game", axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Party Game']==1].index)
games.loc[indices, 'Cat:Party']=1
subcategories.drop('Party Game', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Abstract Strategy']==1].index)
games.loc[indices, 'Cat:Abstract']=1
subcategories.drop('Abstract Strategy', axis=1, inplace=True)

In [44]:
# make new data frames for THEMES and SUBCATEGORIES
themes = subcategories[subcats_themes]
subcategories = subcategories[subcats_subcategories]

In [40]:
# reduce all flags to int8

temp_id = mechanics['BGGId']
mechanics.drop('BGGId', axis=1, inplace=True)
mechanics = mechanics.astype('int8')
mechanics['BGGId'] = temp_id

temp_id = themes['BGGId']
themes.drop('BGGId', axis=1, inplace=True)
themes = themes.astype('int8')
themes['BGGId'] = temp_id

temp_id = subcategories['BGGId']
subcategories.drop('BGGId', axis=1, inplace=True)
subcategories = subcategories.astype('int8')
subcategories['BGGId'] = temp_id

Save all the file we just cleaned or created!

In [41]:
mechanics.to_pickle('data_cleaned/mechanics.pkl')

In [42]:
subcategories.to_pickle('data_cleaned/subcategories.pkl')

In [43]:
themes.to_pickle('data_cleaned/themes.pkl')

In [44]:
games.to_pickle('data_cleaned/games.pkl')

## Designers

In [ ]:
# Load up our designers file!
designers = pd.read_pickle('data_dirty/designers.pkl')
designers.info()

In [ ]:
designers.drop_duplicates(subset='BGGId', keep='first', inplace=True) # drop duplicates
designers.fillna(0, inplace=True) # Fill NaN

# change all flags to int8
temp_id = designers['BGGId'] 
designers.drop('BGGId', axis=1, inplace=True)
designers = designers.astype('int8')
designers['BGGId'] = temp_id

designers.drop(designers.index[empty_games], inplace=True) # drop missing games
designers.reset_index(inplace=True, drop=True) # reset index
designers.drop(designers.index[not_released], inplace=True) # drop unreleased games
designers.reset_index(inplace=True, drop=True)# reset index
designers.info() # check memory usage

In [ ]:
# save file of all possible designers
designers.to_pickle('data_cleaned/designers_all.pkl')

In [ ]:
# locate all row that sum to 3 or less (find low experience designers)

# change the 3 to whatever desired for more or less experience
lowexp_rows = designers.loc[:, designers.sum(axis=0) <= 3]

# Locate the columns that contain the low experience designers
lowexp_columns = lowexp_rows[lowexp_rows.sum(axis=1) > 0]

# get indices of those low-exp columns
indices = lowexp_columns.index

# make new column for low exp designer
designers['Low-Exp Designer'] = 0

# for each index in the low exp list, set low exp designer to 1
for index in indices:
    designers.loc[index, 'Low-Exp Designer'] = 1
    
# drop all columns for one-off designers
designers.drop(designers.loc[:, designers.sum(axis=0) <= 3], axis=1, inplace=True)

In [ ]:
# save file of designers reduced to 3 or more works
designers.to_pickle('data_cleaned/designers_reduced.pkl')

## Artists

In [ ]:
# load artists file
artists = pd.read_pickle('data_dirty/artists.pkl')
artists.info()

In [ ]:
artists.drop_duplicates(subset='BGGId', keep='first', inplace=True) # drop duplicates
artists.fillna(0, inplace=True) # fill nan

# convert flags to int8
temp_id = artists['BGGId']
artists.drop('BGGId', axis=1, inplace=True)
artists = artists.astype('int8')
artists['BGGId'] = temp_id

artists.drop(artists.index[empty_games], inplace=True) # drop empty
artists.reset_index(inplace=True, drop=True) # reset index
artists.drop(artists.index[not_released], inplace=True) # drop unreleased
artists.reset_index(inplace=True, drop=True)# reset index
artists.info()# get memory usage

In [ ]:
# save all artists to file
artists.to_pickle('data_cleaned/artists_all.pkl')

In [ ]:
# locate all row that sum to 3 or less (find low experience artists)

# change the 3 to whatever desired for more or less experience
lowexp_rows = artists.loc[:, artists.sum(axis=0) <= 3]

# Locate the columns that contain the low experience artists
lowexp_columns = lowexp_rows[lowexp_rows.sum(axis=1) > 0]

# get indices of those low-exp columns
indices = lowexp_columns.index

# make new column for low exp Artist
artists['Low-Exp Artist'] = 0

# for each index in the low exp list, set low exp Artist to 1
for index in indices:
    artists.loc[index, 'Low-Exp Artist'] = 1
    
# drop all columns for one-off artists
artists.drop(artists.loc[:, artists.sum(axis=0) <= 3], axis=1, inplace=True)

In [ ]:
# save artists to file reduces to 3 or more works
artists.to_pickle('data_cleaned/artists_reduced.pkl')

## Publishers

In [ ]:
# load publishers
publishers = pd.read_pickle('data_dirty/publishers.pkl')
publishers.info()

In [ ]:
publishers.drop_duplicates(subset='BGGId', keep='first', inplace=True) # drop duplicates
publishers.fillna(0, inplace=True)# fill nan

# set flags to int8
temp_id = publishers['BGGId']
publishers.drop('BGGId', axis=1, inplace=True)
publishers = publishers.astype('int8')
publishers['BGGId'] = temp_id

publishers.drop(publishers.index[empty_games], inplace=True) # drop empty
publishers.reset_index(inplace=True, drop=True) #reset index
publishers.drop(publishers.index[not_released], inplace=True) # drop unreleased
publishers.reset_index(inplace=True, drop=True) # reset index
publishers.info() # get memory usage

In [ ]:
# save all publishers to file
publishers.to_pickle('data_cleaned/publishers_all.pkl')

In [ ]:
# locate all row that sum to 3 or less (find low experience publishers)

# change the 3 to whatever desired for more or less experience
lowexp_rows = publishers.loc[:, publishers.sum(axis=0) <= 3]

# Locate the columns that contain the low experience publishers
lowexp_columns = lowexp_rows[lowexp_rows.sum(axis=1) > 0]

# get indices of those low-exp columns
indices = lowexp_columns.index

# make new column for low exp Publisher
publishers['Low-Exp Publisher'] = 0

# for each index in the low exp list, set low exp Publisher to 1
for index in indices:
    publishers.loc[index, 'Low-Exp Publisher'] = 1
    
# drop all columns for one-off publishers
publishers.drop(publishers.loc[:, publishers.sum(axis=0) <= 3], axis=1, inplace=True)

In [ ]:
# save publishers reduced to 3 or more works
publishers.to_pickle('data_cleaned/publishers_reduced.pkl')

## Awards

In [ ]:
# load awards
awards = pd.read_pickle('data_dirty/awards.pkl')
awards.info()

In [ ]:
awards.drop_duplicates(subset='BGGId', keep='first', inplace=True) # drop duplicates
awards.fillna(0, inplace=True) # fill nan

# reduce flags to int8
temp_id = awards['BGGId']
awards.drop('BGGId', axis=1, inplace=True)
awards = awards.astype('int8')
awards['BGGId'] = temp_id

awards.drop(awards.index[empty_games], inplace=True) # drop empty
awards.reset_index(inplace=True, drop=True) # reset index
awards.drop(awards.index[not_released], inplace=True) # drop unreleased
awards.reset_index(inplace=True, drop=True) # reset index
awards.info() # get memory usage

In [ ]:
# save all awards
awards.to_pickle('data_cleaned/awards_all.pkl')

In [ ]:
# locate all row that sum to 3 or less (find low experience awards)

# change the 3 to whatever desired for more or less experience
lowexp_rows = awards.loc[:, awards.sum(axis=0) <= 20]

# Locate the columns that contain the low experience awards
lowexp_columns = lowexp_rows[lowexp_rows.sum(axis=1) > 0]

# get indices of those low-exp columns
indices = lowexp_columns.index

# make new column for low exp designer
awards['Other Award(s)'] = 0

# for each index in the low exp list, set low exp designer to 1
for index in indices:
    awards.loc[index, 'Other Award(s)'] = 1
    
# drop all columns for one-off awards
awards.drop(awards.loc[:, awards.sum(axis=0) <= 20], axis=1, inplace=True)

In [ ]:
# save awards reduced to 20
awards.to_pickle('data_cleaned/awards_reduced.pkl')

## Ratings Dist

In [ ]:
# open ratings distribution
ratings_dist = pd.read_pickle('data_dirty/ratings_dist.pkl')
ratings_dist.info()

In [ ]:
ratings_dist.drop_duplicates(subset='BGGId', keep='first', inplace=True) # drop duplicates
ratings_dist.fillna(0, inplace=True) # fill nan
ratings_dist.drop(ratings_dist.index[empty_games], inplace=True) # drop empty games
ratings_dist.reset_index(inplace=True, drop=True) # reset index
ratings_dist.drop(ratings_dist.index[not_released], inplace=True) # drop unreleased games
ratings_dist.reset_index(inplace=True, drop=True) # reset index

# for each column:
for column in ratings_dist.columns[1:]:
    # fix numbers as needed (convert 1xk to 10000)
    ratings_dist[column] = ratings_dist[column].apply(lambda x: fix_numbers(x))

# add a column
ratings_dist['num_votes'] = games['NumUserRatings']

# convert types to int32
temp_id = ratings_dist['BGGId']
ratings_dist.drop('BGGId', axis=1, inplace=True)
ratings_dist = ratings_dist.astype('int32')
ratings_dist['BGGId'] = temp_id

ratings_dist.info() # check memory usage

In [ ]:
# save to file
ratings_dist.to_pickle('data_cleaned/ratings_dist.pkl')

## Comments

In [ ]:
# load comments
comments = pd.read_pickle('data_dirty/comments.pkl')
comments.info()

In [ ]:
comments.drop_duplicates(subset=['BGGId', 'Username'], keep='first', inplace=True) # drop duplicates
comments.reset_index(inplace=True, drop=True) # rest index
comments.fillna(0, inplace=True) # fill na

# clean the comments text
comments['cleaned'] = comments['Value'].apply(lambda x: text_block_processor(x))
    
# drop the description field 
comments.drop('Value', axis=1, inplace=True)

In [ ]:
# save file
comments.to_pickle('data_cleaned/comments.pkl')

## Ratings Matrix

In [7]:
# create a ratings matrix file from a set of files on disk
ratings_matrix = create_ratings_file(51,57, game_ids_list)


Cleaning Frame #51
<class 'pandas.core.frame.DataFrame'>
Index: 3757 entries, pfglenn to rototom
Columns: 19731 entries, 178944 to 248188
dtypes: float64(19731)
memory usage: 565.6+ MB
None

Cleaning Frame #52
<class 'pandas.core.frame.DataFrame'>
Index: 1326 entries, rossandjuliette to scottishzombie
Columns: 15292 entries, 1275 to 113995
dtypes: float64(15292)
memory usage: 154.7+ MB
None

Cleaning Frame #53
<class 'pandas.core.frame.DataFrame'>
Index: 2341 entries, scott3387 to soupajoe
Columns: 17040 entries, 68448 to 104029
dtypes: float64(17040)
memory usage: 304.4+ MB
None

Cleaning Frame #54
<class 'pandas.core.frame.DataFrame'>
Index: 1522 entries, soupmanx to swoopaloo
Columns: 15185 entries, 290468 to 40656
dtypes: float64(15185)
memory usage: 176.3+ MB
None

Cleaning Frame #55
<class 'pandas.core.frame.DataFrame'>
Index: 3011 entries, sword18 to turbocooler
Columns: 18457 entries, 39862 to 8132
dtypes: float64(18457)
memory usage: 424.0+ MB
None

Cleaning Frame #56
<class 

In [8]:
# check and discard duplicate rows
ratings_matrix = ratings_matrix[~ratings_matrix.index.duplicated(keep='first')]

In [9]:
# save file to pickle, specify filename carefully!
ratings_matrix.to_pickle('data_cleaned/ratings_matrix_cleaned_06.pkl')

In [10]:
del ratings_matrix
gc.collect()

42